In [5]:
import pandas as pd
import urllib
import urllib.request
import json

In [6]:
servicekey = 's+fQ9LDUrt9xJ9LSIp0R4+gJBR7eOiUpRHNKXMb6gaV844FL4oI+OYVOY+MC2Bff+Iq9bQWFeWrktswAfBtkyg=='
url= 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'

queryParams = '?' + urllib.parse.urlencode(
    {
        urllib.parse.quote_plus('servicekey') : servicekey,
        urllib.parse.quote_plus('pageNo') : '1',
        urllib.parse.quote_plus('numOfRows') : '372',  # 12 * 31 = 372
       # 아님 12개 항목임
        urllib.parse.quote_plus('dataType') : 'JSON', # JSON, XML 두가지 포멧을 제공합니다.
        urllib.parse.quote_plus('base_date') : '20240208', # 예보 받을 날짜를 입력합니다. 최근 1일간의 자료만 제공합니다.
        urllib.parse.quote_plus('base_time') : '1700', # 예보 시간을 입력합니다. 2시부터 시작하여 3시간 단위로 입력 가능합니다.
        urllib.parse.quote_plus('nx') : '48', # 울산 태양광 발전소 x 좌표입니다. '기상청18_동네예보 조회서비스_오픈API활용가이드.zip'에 포함 된 excel파일을 통해 확인 가능합니다.
        urllib.parse.quote_plus('ny') : '36' # 울산 태양광 발전소 y 좌표입니다. '기상청18_동네예보 조회서비스_오픈API활용가이드.zip'에 포함 된 excel파일을 통해 확인 가능합니다.
    }
)

response = urllib.request.urlopen(url + queryParams).read()
response = json.loads(response)

In [7]:
response

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'},
  'body': {'dataType': 'JSON',
   'items': {'item': [{'baseDate': '20240208',
      'baseTime': '1700',
      'category': 'TMP',
      'fcstDate': '20240208',
      'fcstTime': '1800',
      'fcstValue': '5',
      'nx': 48,
      'ny': 36},
     {'baseDate': '20240208',
      'baseTime': '1700',
      'category': 'UUU',
      'fcstDate': '20240208',
      'fcstTime': '1800',
      'fcstValue': '0.7',
      'nx': 48,
      'ny': 36},
     {'baseDate': '20240208',
      'baseTime': '1700',
      'category': 'VVV',
      'fcstDate': '20240208',
      'fcstTime': '1800',
      'fcstValue': '-3.8',
      'nx': 48,
      'ny': 36},
     {'baseDate': '20240208',
      'baseTime': '1700',
      'category': 'VEC',
      'fcstDate': '20240208',
      'fcstTime': '1800',
      'fcstValue': '350',
      'nx': 48,
      'ny': 36},
     {'baseDate': '20240208',
      'baseTime': '1700',
      'category': 'WSD',
      'fcst

In [8]:
import pandas as pd

# 새로운 데이터프레임을 생성
fcst_df = pd.DataFrame(columns=['Forecast_date', 'Forecast_hour', 'WindDirection', 'WindSpeed', 'Cloud', 'Rainfall', 'Humidity', 'Temperature'])

row_idx = 0  # row_idx 초기화

for data in response['response']['body']['items']['item']:
    fcst_df.loc[row_idx, 'Forecast_date'] = data['fcstDate']
    fcst_df.loc[row_idx, 'Forecast_hour'] = data['fcstTime']

    if data['category'] == 'REH':
        fcst_df.loc[row_idx, 'Humidity'] = float(data.get('fcstValue', 'NaN'))
    elif data['category'] == 'PCP':
        fcst_df.loc[row_idx, 'Rainfall'] = str(data.get('fcstValue', 'NaN'))
    elif data['category'] == 'TMP':
        fcst_df.loc[row_idx, 'Temperature'] = float(data.get('fcstValue', 'NaN'))
    elif data['category'] == 'SKY':
        fcst_df.loc[row_idx, 'Cloud'] = float(data.get('fcstValue', 'NaN'))
    elif data['category'] == 'VEC':
        fcst_df.loc[row_idx, 'WindDirection'] = float(data.get('fcstValue', 'NaN'))
    elif data['category'] == 'WSD':
        fcst_df.loc[row_idx, 'WindSpeed'] = float(data.get('fcstValue', 'NaN'))
        row_idx += 1  # 다음 행으로 이동

fcst_df


,Forecast_date,Forecast_hour,WindDirection,WindSpeed,Cloud,Rainfall,Humidity,Temperature
0,20240208,1800,350.0,3.9,NaN,NaN,NaN,5.0
1,20240208,1900,346.0,3.8,4.0,강수없음,65.0,5.0
2,20240208,2000,349.0,3.7,4.0,강수없음,65.0,5.0
3,20240208,2100,355.0,3.7,4.0,강수없음,65.0,5.0
4,20240208,2200,3.0,3.3,4.0,강수없음,65.0,5.0
5,20240208,2300,360.0,3.1,4.0,강수없음,70.0,4.0
6,20240209,0000,354.0,3.1,4.0,강수없음,70.0,4.0
7,20240209,0100,349.0,3.1,4.0,강수없음,70.0,5.0
8,20240209,0200,345.0,3.0,4.0,강수없음,70.0,5.0
9,20240209,0300,342.0,3.3,4.0,강수없음,70.0,5.0
